In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # disable GPU devices
os.environ["TFDS_DATA_DIR"] = os.path.expanduser("~/tensorflow_datasets")  # default location of tfds database
os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
from keras import layers, models, metrics, losses
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

import tensorflow as tf
import tensorflow_datasets as tfds

import numpy as np

import json
from pathlib import Path

import matplotlib.pyplot as plt

# Turn off logging for TF
import logging
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
tf.get_logger().setLevel(logging.ERROR)

from dpmhm.datasets import preprocessing, feature, utils, transformer

ds_all, ds_info = tfds.load(
    'CWRU',
    with_info=True,
)

ds0 = ds_all['train']
ds0.element_spec

2024-07-02 14:59:08.356083: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-02 14:59:08.362178: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-02 14:59:08.471386: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-02 14:59:10.064093: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-07-02 14:59:13.327519: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-02 14:59:13.327593: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] ret

{'metadata': {'Dataset': TensorSpec(shape=(), dtype=tf.string, name=None),
  'FaultComponent': TensorSpec(shape=(), dtype=tf.string, name=None),
  'FaultLocation': TensorSpec(shape=(), dtype=tf.string, name=None),
  'FaultSize': TensorSpec(shape=(), dtype=tf.float32, name=None),
  'FileName': TensorSpec(shape=(), dtype=tf.string, name=None),
  'LoadForce': TensorSpec(shape=(), dtype=tf.uint32, name=None),
  'NominalRPM': TensorSpec(shape=(), dtype=tf.uint32, name=None),
  'RPM': TensorSpec(shape=(), dtype=tf.uint32, name=None)},
 'sampling_rate': TensorSpec(shape=(), dtype=tf.uint32, name=None),
 'signal': {'BA': TensorSpec(shape=(None,), dtype=tf.float32, name=None),
  'DE': TensorSpec(shape=(None,), dtype=tf.float32, name=None),
  'FE': TensorSpec(shape=(None,), dtype=tf.float32, name=None)}}

In [2]:
outdir = Path('/volatile/home/bm279471/tmp/few_shot_cwru')
os.makedirs(outdir, exist_ok=True)

# compactor = transformer.DatasetCompactor(ds0,
#                                          channels=['DE', 'FE', 'BA'],
#                                          keys=['FaultLocation', 'FaultComponent', 'FaultSize'],
#                                          resampling_rate=12000)

# # Feature extractor
# # Spectrogram is computed on a time window of 0.025 second every 0.0125 second, then converted to decibel scale.
# _func = lambda x, sr: feature.spectral_features(x, sr, 'spectrogram',
# #                                                 n_mfcc=256,
#                                                 time_window=0.025, hop_step=0.0125, n_fft=512,
#                                                 normalize=False, to_db=True)[0]

# extractor = transformer.FeatureExtractor(compactor.dataset, _func)

# # A window of width w correspond to w*0.0125 seconds
# window = transformer.WindowSlider(extractor.dataset, window_size=(64,64), hop_size=(32,32))
# # window = transformer.WindowSlider(extractor.dataset, window_size=(256, 80), hop_size=40)  # 1s, full bandwidth
# # window = transformer.WindowSlider(extractor.dataset, window_size=64, hop_size=32)

# labels = list(compactor.full_label_dict.keys())

# preproc = preprocessing.get_mapping_supervised(labels)
    
# ds_window = window.dataset.map(preproc, num_parallel_calls=tf.data.AUTOTUNE)

# eles = list(ds_window.take(10).as_numpy_iterator())
# input_shape = eles[0][0].shape

# ds_window = ds_window.map(lambda x,y: (tf.ensure_shape(x, input_shape), y), num_parallel_calls=tf.data.AUTOTUNE)

# splits = {'train':0.7, 'transfer':0.1,'val':0.1, 'test':0.1}
# ds_split = utils.split_dataset(ds_window, splits, labels=[i+1 for i in range(len(labels))])

# ds_split['train'].save(str(outdir/'ds_train'))
# ds_split['val'].save(str(outdir/'ds_val'))
# ds_split['transfer'].save(str(outdir/'ds_transfer'))
# ds_split['test'].save(str(outdir/'ds_test'))

# with open(outdir/'lb.json', 'w') as fp:
#     json.dump(labels,fp)

In [3]:
ds_train = tf.data.Dataset.load(str(outdir/'ds_train'))
ds_val = tf.data.Dataset.load(str(outdir/'ds_val'))
ds_transfer = tf.data.Dataset.load(str(outdir/'ds_transfer'))
ds_test = tf.data.Dataset.load(str(outdir/'ds_test'))

with open(outdir/'lb.json', 'r') as fp:
    labels = list(json.load(fp))

In [4]:
batch_size = 32
ds_size = utils.get_dataset_size(ds_train) + utils.get_dataset_size(ds_val) +utils.get_dataset_size(ds_transfer) + utils.get_dataset_size(ds_test)
n_embedding  = 128 
kernel_size = (3,3)
projection_dim = 128
nb_classes=len(labels)

2024-07-02 14:59:17.187310: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-07-02 14:59:19.386263: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-07-02 14:59:20.018620: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-07-02 14:59:20.659158: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [5]:
ds_train = ds_train.shuffle(ds_size, reshuffle_each_iteration=True).cache().batch(batch_size,drop_remainder=True).prefetch(tf.data.AUTOTUNE)
ds_val = ds_val.batch(batch_size,drop_remainder=True)
ds_transfer=ds_transfer.shuffle(ds_size, reshuffle_each_iteration=True).cache().batch(batch_size,drop_remainder=True).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(1)

eles = list(ds_train.take(1).as_numpy_iterator())
input_shape = eles[0][0][0].shape

2024-07-02 14:59:22.400424: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-07-02 14:59:22.406911: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [6]:
tf.config.run_functions_eagerly(True)

@tf.keras.utils.register_keras_serializable()
class AddPositionEmbs(layers.Layer):
    """Adds (optionally learned) positional embeddings to the inputs."""

    def build(self, input_shape):
        assert (
            len(input_shape) == 3
        ), f"Number of dimensions should be 3, got {len(input_shape)}"
        self.pe = tf.Variable(
            name="pos_embedding",
            initial_value=tf.random_normal_initializer(stddev=0.06)(
                shape=(1, input_shape[1], input_shape[2])
            ),
            dtype="float32",
            trainable=True,
        )

    def call(self, inputs):
        return inputs + tf.cast(self.pe, dtype=inputs.dtype)

    def get_config(self):
        config = super().get_config()
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

@tf.keras.utils.register_keras_serializable()
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, *args, num_heads, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_heads = num_heads

    def build(self, input_shape):
        embedding_dim = input_shape[-1]
        num_heads = self.num_heads
        if embedding_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embedding_dim} should be divisible by number of heads = {num_heads}"
            )
        self.embedding_dim = embedding_dim
        self.projection_dim = embedding_dim // num_heads
        self.query_dense = layers.Dense(embedding_dim, name="query")
        self.key_dense = layers.Dense(embedding_dim, name="key")
        self.value_dense = layers.Dense(embedding_dim, name="value")
        self.combine_heads = layers.Dense(embedding_dim, name="out")

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], score.dtype)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)

        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embedding_dim))
        output = self.combine_heads(concat_attention)
        return output, weights

    def get_config(self):
        config = super().get_config()
        config.update({"num_heads": self.num_heads})
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


@tf.keras.utils.register_keras_serializable()
class TransformerBlock(layers.Layer):
    """Implements a Transformer block."""

    def __init__(self, *args, num_heads, mlp_dim, dropout, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_heads = num_heads
        self.mlp_dim = mlp_dim
        self.dropout = dropout

    def build(self, input_shape):
        self.att = MultiHeadSelfAttention(
            num_heads=self.num_heads,
            name="MultiHeadDotProductAttention_1",
        )
        self.mlpblock = keras.Sequential(
            [
                layers.Dense(
                    self.mlp_dim,
                    activation="linear",
                    name=f"{self.name}_Dense_0",
                ),
                layers.Lambda(
                    lambda x: keras.activations.gelu(x, approximate=False)
                )
                if hasattr(keras.activations, "gelu")
                else layers.Lambda(
                    lambda x: tf.nn.gelu(x, approximate=False)
                ),
                layers.Dropout(self.dropout),
                layers.Dense(input_shape[-1], name=f"{self.name}_Dense_1"),
                layers.Dropout(self.dropout),
            ],
            name="MlpBlock_3",
        )
        self.layernorm1 = layers.LayerNormalization(
            epsilon=1e-6, name="LayerNorm_0"
        )
        self.layernorm2 = layers.LayerNormalization(
            epsilon=1e-6, name="LayerNorm_2"
        )
        self.dropout_layer = layers.Dropout(self.dropout)

    def call(self, inputs, training):
        x = self.layernorm1(inputs)
        x, weights = self.att(x)
        x = self.dropout_layer(x, training=training)
        x = x + inputs
        y = self.layernorm2(x)
        y = self.mlpblock(y)
        return x + y, weights

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "num_heads": self.num_heads,
                "mlp_dim": self.mlp_dim,
                "dropout": self.dropout,
            }
        )
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

@tf.function()
def SampleMaskIndex(seq_len, N, C):
        I = set()
        while len(I) < N:
            i = np.random.randint(0, seq_len)  # uniform distribution over {1, N}
            Ic = set(range(max(0, i-C+1), min(seq_len, i+C)))
            I = I.union(Ic)
        I = list(I)[:N]  # guarantee to mask exactly N patches
        return I

@tf.keras.utils.register_keras_serializable()
class Mask(layers.Layer):
    def __init__(self, mask_ratio=0.15, batch_size=32, embedding_dim=128, seq_len=16, cluster_factor=3, **kwargs):
        super(Mask, self).__init__(**kwargs)
        self.mask_ratio = mask_ratio
        self.batch_size=batch_size
        self.embedding_dim=embedding_dim
        self.seq_len=seq_len
        self.cluster_factor=cluster_factor
        self.learnable_mask = self.add_weight(
            shape=(self.batch_size, seq_len, embedding_dim),
            initializer="glorot_uniform",  
            trainable=True,
            name="learnable_mask",
        )

    def call(self, y):
        N = tf.cast(self.seq_len * self.mask_ratio, tf.int32)

        masked_indices = SampleMaskIndex(self.seq_len, N, self.cluster_factor)

        # Create the mask Tensor and the masked embeddings
        mask = tf.ones((self.batch_size, self.seq_len, self.embedding_dim))

        indices = tf.constant([[i, j] for i in range(self.batch_size) for j in masked_indices])
        updates = tf.zeros((len(indices), self.embedding_dim))

        mask = tf.tensor_scatter_nd_update(mask, indices, updates)
        y_masked = y * mask + (tf.ones((self.batch_size, self.seq_len, self.embedding_dim)) - mask) * self.learnable_mask

        # Create a list of candidates for each masked patch
        candidate_patches = []
        for i in range(self.batch_size):
            batch_candidates=[]
            for j, idx in enumerate(masked_indices):
                # for each masked patch, we introduce the masked patch and then the rest of the masked patches
                patches = []
                patches.append(y[i, idx, :])  # correct index
                for masked_idx in masked_indices[:j] + masked_indices[j+1:]:
                    patches.append(y[i, masked_idx, :])
                batch_candidates.append(patches)
            candidate_patches.append(batch_candidates)

        return y_masked, mask, candidate_patches, masked_indices

@tf.keras.utils.register_keras_serializable()
class SSASTModel(models.Model):
    def __init__(self, image_size=(64,64), patch_size=16, num_layers=3, embedding_dim=128, num_heads=4, mlp_dim=128, dropout=0.1, mask_ratio=0.6, batch_size=32, Lambda=10, cluster_factor=3, **kwargs):
        super(SSASTModel, self).__init__(**kwargs)
        self.seq_len=(image_size[0]//patch_size)*(image_size[1]//patch_size)
        self.image_size = image_size
        self.patch_size = patch_size
        self.num_layers = num_layers
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.mlp_dim = mlp_dim
        self.dropout = dropout
        self.batch_size=batch_size
        self.Lambda=Lambda
        self.mask_ratio=mask_ratio

        self.embedding=layers.Conv2D(
            filters=self.embedding_dim,
            kernel_size=self.patch_size,
            strides=self.patch_size,
            padding="valid",
            name="embedding",
        )
        self.reshape =layers.Reshape((self.seq_len, self.embedding_dim))
        self.position_embedding=AddPositionEmbs(name="Transformer_posembed_input")
        self.mask_layer = Mask(mask_ratio=mask_ratio, batch_size=batch_size, embedding_dim=embedding_dim, seq_len=self.seq_len, cluster_factor=cluster_factor)
        self.transformer=[
            TransformerBlock(
                num_heads=self.num_heads,
                mlp_dim=self.mlp_dim,
                dropout=self.dropout,
                name=f"Transformer_encoderblock_{n}",
            ) for n in range(self.num_layers)
        ]
        self.transformer_norm = layers.LayerNormalization(
            epsilon=1e-6, name="Transformer_encoder_norm"
        )
        self.embedding_inverse = layers.Conv2DTranspose(
            filters=3,
            kernel_size=self.patch_size,
            strides=self.patch_size,
            padding="valid",
            name="embedding_inverse",
        )
        self.reshape_inverse = layers.Reshape((self.image_size[0] // self.patch_size, self.image_size[1] // self.patch_size, self.embedding_dim))

    def reconstruction(self, O, mask):
        '''Reconstructs the original masked patches from the transformer output and computes it to the spectrogram shape'''
        O = (tf.ones((O.shape[0], O.shape[1], O.shape[2])) - mask) * O
        return self.embedding_inverse(self.reshape_inverse(O))
    
    def classification(self, O, mask, candidate_patches, masked_indices):
        """For each masked patch, selects the correct patch among the candidates patches"""
        masked_patches = (tf.ones((mask.shape[0], mask.shape[1], mask.shape[2])) - mask) * O
        classifications = []
        for i, elem in enumerate(masked_patches):
            patch_classifications = []
            for idx, j in enumerate(masked_indices):
                patch = elem[j]
                candidate_similarities = []
                for candidate in candidate_patches[i][idx]:
                    similarity = tf.reduce_sum(patch * candidate, axis=-1)
                    candidate_similarities.append(similarity)
                patch_classification = tf.stack(candidate_similarities)
                patch_classifications.append(patch_classification)
            classifications.append(patch_classifications)
        return classifications

    def convert_list_to_tensor(self, L, dtype=tf.float32):
        max_len = max(len(inner_list) for inner_list in L)
        padded_lists = []
        for inner_list in L:
            padded_list = [tf.convert_to_tensor(item, dtype=dtype) for item in inner_list]
            padded_list += [tf.zeros_like(inner_list[0], dtype=tf.float32)] * (max_len - len(inner_list))
            padded_lists.append(tf.stack(padded_list))
        return tf.stack(padded_lists)

    def call(self, x):
        E = self.embedding(x)
        E = self.reshape(E)

        E_mask, mask, candidate_patches, masked_indices = self.mask_layer.call(E)

        O = self.position_embedding(E_mask)
        for n in range(self.num_layers):
            O, _ = self.transformer[n](O, training=True)
        O = self.transformer_norm(O)

        r_i = self.reconstruction(O, mask)
        c_i = self.classification(O, mask, candidate_patches, masked_indices)

        r_i_true = (tf.ones((self.batch_size, E.shape[1], self.embedding_dim)) - mask) * E
        r_i_true = self.embedding_inverse(self.reshape_inverse(r_i_true))
        c_i_true = [[0 for _ in range(len(masked_indices))] for _ in range(self.batch_size)]  # Set all true indices to 0

        c_i = self.convert_list_to_tensor(c_i)
        c_i_true = tf.convert_to_tensor(c_i_true, dtype=tf.int64)

        c_i = tf.reshape(c_i, [-1, int(self.mask_ratio*self.seq_len)])
        c_i_true = tf.reshape(c_i_true, [-1])

        return r_i, r_i_true, c_i, c_i_true

    def compile(self, optimizer, **kwargs):
        super(SSASTModel, self).compile(**kwargs)
        self.optimizer = optimizer
        self.loss_tracker = metrics.Mean(name="loss")
        self.reconstruction_loss_tracker = metrics.Mean(name="reconstruction_loss")
        self.classification_loss_tracker = metrics.Mean(name="classification_loss")

    @property
    def metrics(self):
        return [self.loss_tracker, self.reconstruction_loss_tracker, self.classification_loss_tracker]

    def train_step(self, data):
        x, _ = data
        with tf.GradientTape() as tape:
            r_i, r_i_true, c_i, c_i_true = self(x, training=True)
            reconstruction_loss = self.Lambda*losses.mean_squared_error(r_i_true, r_i)
            classification_loss = losses.sparse_categorical_crossentropy(c_i_true, c_i, from_logits=True)
            total_loss = tf.reduce_mean(reconstruction_loss) + tf.reduce_mean(classification_loss)

        grads = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))

        self.loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.classification_loss_tracker.update_state(classification_loss)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        x, _ = data

        r_i, r_i_true, c_i, c_i_true = self(x, training=False)
        reconstruction_loss = self.Lambda * losses.mean_squared_error(r_i_true, r_i)
        classification_loss = losses.sparse_categorical_crossentropy(c_i_true, c_i, from_logits=True)
        total_loss = tf.reduce_mean(reconstruction_loss) + tf.reduce_mean(classification_loss)

        self.loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.classification_loss_tracker.update_state(classification_loss)
        return {m.name: m.result() for m in self.metrics}
    
ssast_model = SSASTModel(image_size=(input_shape[0], input_shape[1]), Lambda=1000)
test_input = tf.random.normal((32, input_shape[0], input_shape[1], 3))
test_output = ssast_model(test_input)
print(len(test_output), [o.shape for o in test_output])

4 [TensorShape([32, 64, 64, 3]), TensorShape([32, 64, 64, 3]), TensorShape([288, 9]), TensorShape([288])]


In [7]:
ssast_model.compile(
    optimizer=keras.optimizers.Adam()
)

# early_stopping = EarlyStopping(
#     monitor='val_loss',  
#     patience=3,       
#     restore_best_weights=True  
# )

# reduce_lr = ReduceLROnPlateau(
#     monitor='val_loss', 
#     factor=0.1,         
#     patience=2          
# )

# history = ssast_model.fit(
#     ds_train.repeat(),
#     validation_data=ds_val,
#     epochs=4,
#     steps_per_epoch=(int(0.7*ds_size)//batch_size),
#     callbacks=[early_stopping, reduce_lr]
# )

In [8]:
# ssast_model.save_weights('ssast.weights.h5')
ssast_model.load_weights('ssast.weights.h5')

/volatile/home/bm279471/Documents/.venv/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 112 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [9]:
# loss = history.history['loss']
# val_loss = history.history['val_loss']
# reconstruction_loss = history.history['reconstruction_loss']
# val_reconstruction_loss = history.history['val_reconstruction_loss']
# classification_loss = history.history['classification_loss']
# val_classification_loss = history.history['val_classification_loss']

# # Créer une figure et des axes
# fig, ax = plt.subplots(figsize=(10, 6))

# # Plot loss et val_loss
# epochs = np.arange(1, len(loss) + 1)
# ax.plot(epochs, loss, label='Training Loss', marker='o', linestyle='-', color='r')
# ax.plot(epochs, val_loss, label='Validation Loss', marker='o', linestyle='--', color='r')


# ax.plot(epochs, reconstruction_loss, label='Training Reconstruction Loss', marker='o', linestyle='-', color='b')
# ax.plot(epochs, val_reconstruction_loss, label='Validation Reconstruction Loss', marker='o', linestyle='--', color='b')

# ax.plot(epochs, classification_loss, label='Training Classification Loss', marker='o', linestyle='-', color='g')
# ax.plot(epochs, val_classification_loss, label='Validation Classification Loss', marker='o', linestyle='--', color='g')

# # Configurer les labels et la légende
# ax.set_xlabel('Epochs')
# ax.set_ylabel('Loss')
# ax.set_title('Training and Validation Losses')
# ax.legend()

# # Afficher le graphique
# plt.tight_layout()
# plt.show()

In [10]:
@tf.keras.utils.register_keras_serializable()
class ClassificationModel(models.Model):
    def __init__(self, model, **kwargs):
        super(ClassificationModel, self).__init__(**kwargs)
        self.seq_len = model.seq_len
        self.image_size = model.image_size
        self.patch_size = model.patch_size
        self.num_layers = model.num_layers
        self.embedding_dim = model.embedding_dim
        self.num_heads = model.num_heads
        self.mlp_dim = model.mlp_dim
        self.dropout = model.dropout
        self.batch_size = model.batch_size

        self.embedding = model.embedding
        self.reshape = model.reshape
        self.position_embedding = model.position_embedding
        self.transformer = model.transformer
        self.transformer_norm = model.transformer_norm
        self.meanpooling = layers.GlobalAveragePooling1D(name="MeanPooling")
        self.classification_layer = layers.Dense(nb_classes, name="Classification_head")

    def _set_non_trainable_layers(self):
        for layer in [self.embedding, self.reshape, self.position_embedding, self.transformer_norm]:
            layer.trainable = False
        for transformer in self.transformer:
            transformer.trainable = False
            transformer.att.query_dense.trainable = False
            transformer.att.key_dense.trainable = False
            transformer.att.value_dense.trainable = False
            transformer.att.combine_heads.trainable = False
            transformer.mlpblock.trainable = False
            transformer.layernorm1.trainable = False
            transformer.layernorm2.trainable = False
            transformer.dropout_layer.trainable = False

    def _set_trainable_layers(self):
        for layer in [self.embedding, self.reshape, self.position_embedding, self.transformer_norm]:
            layer.trainable = True
        for transformer in self.transformer:
            transformer.trainable = True
            transformer.att.query_dense.trainable = True
            transformer.att.key_dense.trainable = True
            transformer.att.value_dense.trainable = True
            transformer.att.combine_heads.trainable = True
            transformer.mlpblock.trainable = True
            transformer.layernorm1.trainable = True
            transformer.layernorm2.trainable = True
            transformer.dropout_layer.trainable = True

    def call(self, x):
        E = self.embedding(x)
        E = self.reshape(E)

        O = self.position_embedding(E)
        for n in range(self.num_layers):
            O, _ = self.transformer[n](O, training=True)
        O = self.transformer_norm(O)

        O = self.meanpooling(O)
        O = self.classification_layer(O)
        return O
    
classification_model=ClassificationModel(ssast_model)
classification_model._set_non_trainable_layers()

classification_model.summary(show_trainable=True, expand_nested=True)

Model: "classification_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ embedding (Conv2D)          │ ?                     │     98,432 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ reshape (Reshape)           │ ?                     │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ Transformer_posembed_input  │ ?                     │          0 │   -   │
│ (AddPositionEmbs)           │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ Transformer_encoderblock_0  │ ?                     │     99,584 │   N   │
│ (TransformerBlock)          │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ Transformer_encoderblock_1  │ ?                     │     99,584 │   N   │
│ (TransformerBlock)          │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ Transformer_encoderblock_2  │ ?                     │     99,584 │   N   │
│ (TransformerBlock)          │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ Transformer_encoder_norm    │ ?                     │        256 │   N   │
│ (LayerNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ MeanPooling                 │ ?                     │          0 │   -   │
│ (GlobalAveragePooling1D)    │                       │  (unbuilt) │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ Classification_head (Dense) │ ?                     │          0 │   -   │
│                             │                       │  (unbuilt) │       │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 397,440 (1.52 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 397,440 (1.52 MB)

In [11]:
classification_model.compile(
    optimizer=keras.optimizers.Adam(), 
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=['accuracy'])

early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=5,       
    restore_best_weights=True  
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.1,         
    patience=2          
)

classification_history = classification_model.fit(
    ds_transfer.repeat(),
    validation_data=ds_val,
    epochs=40,
    steps_per_epoch=(int(0.1*ds_size)//batch_size),
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/40


50/50 ━━━━━━━━━━━━━━━━━━━━ 32s 627ms/step - accuracy: 0.0567 - loss: 3.3778 - val_accuracy: 0.0862 - val_loss: 3.2245 - learning_rate: 0.0010
Epoch 2/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 26s 517ms/step - accuracy: 0.1150 - loss: 3.1980 - val_accuracy: 0.1019 - val_loss: 3.1152 - learning_rate: 0.0010
Epoch 3/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 26s 519ms/step - accuracy: 0.1269 - loss: 3.1039 - val_accuracy: 0.1244 - val_loss: 3.0468 - learning_rate: 0.0010
Epoch 4/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 25s 510ms/step - accuracy: 0.1440 - loss: 3.0403 - val_accuracy: 0.1456 - val_loss: 2.9961 - learning_rate: 0.0010
Epoch 5/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 26s 529ms/step - accuracy: 0.1647 - loss: 2.9853 - val_accuracy: 0.1488 - val_loss: 2.9535 - learning_rate: 0.0010
Epoch 6/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 26s 531ms/step - accuracy: 0.1762 - loss: 2.9437 - val_accuracy: 0.1644 - val_loss: 2.9190 - learning_rate: 0.0010
Epoch 7/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 26s 514ms/step - accuracy: 0.1854 - loss: 2.9074 - val_

In [12]:
evaluation = classification_model.evaluate(ds_test)
print("Evaluation accuracy : {:.2f}%".format(evaluation[1]*100))

1654/1654 ━━━━━━━━━━━━━━━━━━━━ 187s 113ms/step - accuracy: 0.3295 - loss: 2.4944
Evaluation accuracy : 33.31%


Fine tuning

In [13]:
classification_model._set_trainable_layers()

classification_model.compile(
    optimizer=keras.optimizers.Adam(1e-5), 
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=['accuracy'])

early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=5,       
    restore_best_weights=True  
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.1,         
    patience=2          
)

classification_history = classification_model.fit(
    ds_transfer.repeat(),
    validation_data=ds_val,
    epochs=40,
    steps_per_epoch=(int(0.1*ds_size)//batch_size),
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 45s 907ms/step - accuracy: 0.3978 - loss: 2.3031 - val_accuracy: 0.3619 - val_loss: 2.2379 - learning_rate: 1.0000e-05
Epoch 2/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 39s 785ms/step - accuracy: 0.4324 - loss: 2.0594 - val_accuracy: 0.4075 - val_loss: 2.1034 - learning_rate: 1.0000e-05
Epoch 3/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 33s 659ms/step - accuracy: 0.4794 - loss: 1.8625 - val_accuracy: 0.4356 - val_loss: 1.9598 - learning_rate: 1.0000e-05
Epoch 4/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 35s 712ms/step - accuracy: 0.5151 - loss: 1.6990 - val_accuracy: 0.4669 - val_loss: 1.8509 - learning_rate: 1.0000e-05
Epoch 5/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 35s 712ms/step - accuracy: 0.5636 - loss: 1.5735 - val_accuracy: 0.4906 - val_loss: 1.7584 - learning_rate: 1.0000e-05
Epoch 6/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 36s 713ms/step - accuracy: 0.5900 - loss: 1.4892 - val_accuracy: 0.5100 - val_loss: 1.6857 - learning_rate: 1.0000e-05
Epoch 7/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 35s 709ms/step - acc

In [14]:
evaluation = classification_model.evaluate(ds_test)
print("Evaluation accuracy : {:.2f}%".format(evaluation[1]*100))

1654/1654 ━━━━━━━━━━━━━━━━━━━━ 99s 60ms/step - accuracy: 0.8055 - loss: 0.7815
Evaluation accuracy : 81.98%
